In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
from pymystem3 import Mystem
m = Mystem()

In [2]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [3]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [4]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [5]:
data = pd.concat([pd.read_json(file, lines=True,  encoding='utf-8') for file in files][:1], axis=0, ignore_index=True)

In [6]:
data.shape

(999, 5)

In [8]:
data.head(1)

,content,keywords,summary,title,url
0,"Многие интересуются, зачем нужна «Яблоку» молодежная фракция? Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах. Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.\nМы ведем борьбу с обязательным воинским призывом. Военный – это профессия, а не обязанность. Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали. По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.\nТакже на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.\nРасскажу о московских активистах. Виктору Петрунину – 19 лет, он пришел к нам боль...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельность становится опасной",http://www.ng.ru/ng_politics/2017-04-18/11_6976_apple.html


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [9]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [10]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [10]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.06
F1 -  0.06
Jaccard -  0.03


Или 10.

In [11]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.07
F1 -  0.06
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [12]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [13]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.0
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [14]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                         ["молодежное, "яблоко":, оппозиционная, деятельность, становится, опасной]
1                                                                 ["газпрома", на, всех, не, хватит]
2                                                   [бесконечная, партия, в, четырехмерные, шахматы]
3    [экс-депутат,, осужденная, за, фальсификацию, выборов,, оказалась, членом, "боевого, братства"]
4                               [новая, москва, останется, территорией, экологической, безопасности]
5                                [f1., гран-при, сша, прошел, без, четырех, машин, и, со, «стопкой»]
6                                          [100, ведущих, политиков, россии, в, феврале, 2018, года]
7                                               [закон, "о, культуре", принимают, на, фоне, арестов]
8                                    [насколько, реальна, газовая, подоплека, сирийского, конфликта]
9                                  [фсб:, в, калужской, области, задержаны, четверо, участн

In [15]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                                                      [в, и, на, не, что, –, его, «молодежное, с, это]
1                                                            [в, и, на, –, млрд., куб., по, к, газа, м]
2                                                                 [в, –, и, не, я, но, что, это, на, с]
3                                                       [в, на, и, ким, по, –, что, видео, он, зинаиды]
4                                              [в, и, на, новой, площадью, москвы, –, развития, с, для]
5                                                             [в, на, и, не, с, но, уже, что, у, гонки]
6                                                  [на, в, (с, место)., и, рф, позиции, влияние, по, с]
7                                                        [в, и, –, по, с, культуре, не, из, будет, как]
8                                                              [в, и, на, с, что, для, по, –, не, газа]
9    [в, рф, террористической, организации, задержаны, –, четвер

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [26]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [27]:
data['content_norm'] = data['content'].apply(normalize)

In [28]:
data['title_norm'] = data['title'].apply(normalize)

In [29]:
data['title_norm'].head(10)

0            [молодёжный, яблоко, оппозиционный, деятельность, становиться, опасный]
1                                                                 [газпром, хватить]
2                                      [бесконечный, партия, четырехмерный, шахматы]
3    [экс-депутат, осудить, фальсификация, выбор, оказаться, член, боевой, братство]
4                 [новый, москва, остаться, территория, экологический, безопасность]
5                         [f1, гран-при, сша, пройти, четыре, машина, стопка, штраф]
6                                [100, ведущий, политик, россия, февраль, 2018, год]
7                                           [закон, культура, принимать, фон, арест]
8                     [насколько, реальный, газовый, подоплёка, сирийский, конфликт]
9                       [фсб, калужский, область, задержать, четверо, участник, иго]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [30]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


***Эксперимент 1***  
Используем лемматизацию от mystem вместо pymorphy

In [ ]:
# m = Mystem(mystem_bin='C://path_to_preloaded/mystem.exe')

In [62]:
def normalize2(text):    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [m.lemmatize(word) for word in words]
    words = [word[0] for word in words if word]
    return words

In [63]:
a = normalize2("Колобок, съел лису?")
a

['колобок', 'съедать', 'лиса']

Само по себе работает. Но когда запускаем (неважно подавая пословно или по предложениям), работа mystem никогда не завершается. Видно, что по каждому вызову создается отдельный процесс mystem.exe, после чего закрывается. Естественно, что это очень медленно.

In [64]:
data['content_norm2'] = data['content'].apply(normalize2)

KeyboardInterrupt: 

**Результат 1: неудачно**

***Эксперимент 2***  
Используем модуль multi-rake, для которого декларируется поддержка русского языка

In [21]:
from multi_rake import Rake

rake = Rake(
    min_chars=3,
    max_words=2,
    min_freq=1,
    language_code='ru',  # 'en'
    stopwords=stops,  # {'and', 'of'}
    lang_detect_threshold=50,
    max_words_unknown_lang=2,
    generated_stopwords_percentile=80,
    generated_stopwords_max_len=3,
    generated_stopwords_min_freq=2,
)

In [22]:
#keywords = rake.apply(text, text_for_stopwords=other_text)
def lel(txt):
    print(txt)
    return
data['content_norm_text'] = data['content_norm'].apply(" ".join)

In [23]:
data['content_rake'] = data['content_norm_text'].apply(rake.apply)

In [24]:
data['content_rake']

0                                                                                                                                  []
1                                                                                                                                  []
2                                                                                                         [(огорчительный факт, 4.0)]
3                                                                                                                                  []
4                                                                                                                                  []
5                                                                                     [(феттель вместе, 4.0), (перес результат, 4.0)]
6                                                                                                                                  []
7                                                             

***Вывод***: результат ужасный   
**Результат 2: неудачно**

***Эксперимент 3***  
NLTK Rake


In [31]:
from rake_nltk import Metric, Rake as NltkRake

nRake = NltkRake(language='russian', stopwords=stops, min_length=1, max_length=2) 

def procRake(txt):
    # print(txt +"\n")    
    nRake.extract_keywords_from_text(txt)
    if nRake:
        print(nRake.get_ranked_phrases())
    return nRake.get_ranked_phrases()
    


**Пробуем NLTK Rake по нормализованным текстам**

In [32]:
data['content_norm_text'].apply(procRake)

[]
['вр новатэк']
['хороший довольный', 'семья говорящий', 'русски плохо', 'огорчительный факт', 'россия', 'писать', 'метр']
[]
['гигант спортивно']
['феттель вместе', 'перес результат', 'авария спровоцировать', 'пит', 'лейный']
['ть 82', 'российский торгово', 'е место', 'го 98', 'го 96', 'го 92', 'го 9', 'го 86', 'го 83', 'го 8', 'го 78', 'го 77', 'го 76', 'го 74', 'го 72', 'го 71', 'го 68', 'го 66', 'го 63', 'го 62', 'го 61', 'го 60', 'го 58', 'го 56', 'го 53', 'го 48', 'го 46', 'го 45', 'го 43', 'го 42', 'го 41', 'го 40', 'го 38', 'го 3', 'го 29', 'го 20', 'го 15', 'го 11']
['д разработать']
['центр russia', 'издание www', 'зенитный комплекс', 'российско', 'план', 'намерение', 'heise']
[]
['половинка термо', 'неподцензурный город', 'банка бросить', 'таблетка', 'старший', 'садиться']
['метр 2020', '6']
[]
['сахалинск москва', 'план жюри', 'вынужденный общаться', 'е', '40']
['авторы']
[]
[]
['половина 2016', 'поколение 85', 'кобальт четыре', '6 миллион', '3 миллион', '1 миллион', '1',

['центр russia', 'издание www', 'зенитный комплекс', 'российско', 'план', 'намерение', 'heise']
['процесс внутри', 'заключить договорённость', 'хау']
[]
['пг', 'мон', '08', '073']
['осуществлять контроль', 'эз']
[]
['именно средство', 'вынудить ввести', '2 1166', 'расширение']
['добавить']
[]
[]
[]
['попытка 96', 'непохожий открывать', 'вблизи видно', 'человек', 'отличный']
[]
['обстоятельство']
['ха год', 'часть', 'тахрир']
[]
['управлять страна', 'метр']
['завысить послабить']
[]
['утверждать']
['столица направление', 'вице']
['читаться конспект', 'читательский радость', 'оригинальный это', 'жизнь сохранение', 'го стать', 'важный причина', 'коренной книга', 'архетипический книга', 'книга', 'глотаться']
['колоссальный']
[]
[]
['губернаторов', 'вице']
['результат 1']
[]
['ха год', 'пропагандист стать', 'младший будущее', 'метр 80', 'метр 60', 'втянутый многие', 'участник', 'лидер', '50']
['образ видеть', 'либо писать', 'строчка', 'основание']
[]
['исследовательский опытно', 'пек']
[]
[

0                                                                                                                                                                                                                                                                                          []
1                                                                                                                                                                                                                                                                                [вр новатэк]
2                                                                                                                                                                                                [хороший довольный, семья говорящий, русски плохо, огорчительный факт, россия, писать, метр]
3                                                                                                                                             

Результат бредовый

**Результат 3: неудачно**
По ненормализованным данным получаем кучу непонятных словосочетаний, среди которых встречается что-то толковое, но в целом ***неудовлетворительно***.

In [33]:
data['content'].apply(procRake)

['функцию организатора', 'федеральном уровне', 'учебных заведений', 'убийства закрыто', 'тех регионов', 'такую ответственность', 'судебный процесс', 'стороны силовиков', 'стараемся координировать', 'становимся кандидатами', 'сих пор', 'российской экономике', 'различных сми', 'проходят лекции', 'провести акцию', 'предлагать взять', 'права женщин', 'поступали угрозы', 'поставленными задачами', 'пассивное наблюдение', 'открытость политики', 'освобождение политзаключенных', 'огромное влияние', 'обсуждения социально', 'образом действовать', 'неподдельная заинтересованность', 'необходимо каким', 'некоторые пытаются', 'некоторое время', 'нашу форму', 'наступления консерватизма', 'наемные работники', 'московских активистах', 'московские активисты', 'многие интересуются', 'многие боятся', 'мнению одного', 'либеральной альтернативы', 'контрактную армию', 'команде единомышленников', 'избегая насилия', 'идеологически силе', 'заканчивая школу', 'единого руководства', 'должен присоединиться', 'деяте

['этому автодрому', 'шмитц выросла', 'сабину вовсе', 'принимала участие', 'нюрбургринге 15', 'кольцевыми автогонкам', 'германии пройдут', 'время дождя', 'включая телесюжеты', 'автоспорте ».', '16 мая', 'fia wtcc', 'wtcc', 'числе', 'соцсетях', 'соревнования', 'сказать', 'пугает', 'правило', 'печально', 'знаменитой', 'гонке', 'год', '45']
['яны вагнер', 'читая отрывки', 'хорошей литературе', 'французском языке', 'французский язык', 'сфере культурного', 'смогли приехать', 'следующнм номере', 'своим творчеством', 'российском представительстве', 'рассказывали друзья', 'призванным свидетельствовать', 'предстоит познакомиться', 'посвященные 100', 'поделившийся воспоминаниями', 'первой половине', 'наталья стеркина', 'музыкальное сопровождение', 'михаил шемякин', 'марии галиной', 'лучший перевод', 'загадках творчества', 'доме брюсова', 'двух томах', 'гуманитарного обмена', 'галина шаталова', 'виктора ремизова', 'виктор коллегорский', 'вечер единомышленников', 'богат событиями', 'алексея макушин

['ходом голосования', 'ходе голосования', 'ухудшился заметно', 'уровень жизни', 'телевизора »,', 'способного представить', 'слоях населения', 'своем сайте', 'результаты голосования', 'результат улучшен', 'путина ».', 'прочих проблемах', 'принято говорить', 'представители которой', 'подготовке выборов', 'плюс работало', 'оппозиционно настроенная', 'оперативное реагирование', 'образцовое наблюдение', 'обработки 100', 'настолько серьезны', 'население видит', 'муниципальными выборами', 'морозный день', 'максим сурайкин', 'либеральной оппозиции', 'которой традиционно', 'имеет поддержку', 'дала президенту', 'власти столица', '96 %)', '72 %).', '7 %.', '60 %.', '5 раза', '2012 году', '08 %,', 'существенных нарушения', 'нарушения недопустимы', 'президентских выборах', 'получив 4', 'голосов избирателей', '« москвичи', 'нарушения', 'голосов', 'выборах', '«', '4', '–', 'явка', 'это', 'экспертов', 'частности', 'холодильник', 'участках', 'урну', 'считаю', 'стране', 'сторонников', 'спасибо', 'состоя

['экономической сфере', 'целью нормы', 'хочется задать', 'филиппинах просидели', 'успело обновиться', 'тормозят развитие', 'тормозит экономический', 'технологический рост', 'такое пребывание', 'таких сущностей', 'сферах жизни', 'стратегических вооружениях', 'старой сущности', 'стандартного набора', 'сохранившейся прежней', 'силовых структур', 'серьезные шаги', 'серьезные опасности', 'свои государства', 'разворачивающийся кризис', 'прекращение переговоров', 'понятное дело', 'положении страны', 'показать явку', 'одной руке', 'одних руках', 'нужно обеспечить', 'новых поступлений', 'новая экономика', 'нашего государства', 'наведаются ко', 'крупные издания', 'исламскими странами', 'информационного пространства', 'засомневался ),', 'закономерное следствие', 'зависимость судов', 'ежедневных газетах', 'единственный кандидат', 'единообразных передовиц', 'единого центра', 'е годы', 'добавить назарбаева', 'дело вкуса', 'действительно находимся', 'данном направлении', 'давно пройдена', 'внутренней

['эта программа', 'эксперты полагают', 'экономических трудностях', 'числа переселенцев', 'федеральном уровне', 'убежден эксперт', 'трудовой патент', 'требуя открыть', 'те приезжие', 'стадии подачи', 'соотечественники выступают', 'сих пор', 'респондентов исследования', 'регионам переезда', 'программа является', 'приходится платить', 'приходит отказ', 'приоритетному переселению', 'принесла свидетельство', 'понравилась печать', 'отметил затулин', 'отмену ограничений', 'одного человека', 'обмен настаивают', 'нынешнем виде', 'незначительность льгот', 'натурализации соотечественников', 'наибольшие неприятности', 'миграционной сфере', 'месяц стоит', 'лучшим вариантом', 'либо заплатить', 'которым отказывают', 'корректировке программ', 'десятков женщин', 'делам снг', 'дальний восток', 'выяснилось также', 'временное проживание', 'авторы жалуются', 'мешали «', '« сколько', 'хотя правительство', 'фпо выяснили', 'сдать документы', 'рф соотечественников', 'регистрации брака', 'общей сложности', '«',

...

['северном кавказе', 'предварительным данным', 'понедельник вечером', 'лесном массиве', 'инцидент произошел', 'инспектор дпс', 'убит', 'стрельбу', 'ранения']
['этим вопросам', 'экологических вопросах', 'числе санно', 'частных застройщиков', 'территориальному планированию', 'таком составе', 'свое письмо', 'расчетами ошиблись', 'принять решение', 'президент призвал', 'право пойти', 'подаче заявки', 'повторил путин', 'переносе объектов', 'отбора подрядчиков', 'окончательная сумма', 'однако чиновники', 'объявлен конкурс', 'началом совещания', 'наибольшая нагрузка', 'муниципальные власти', 'мок ответ', 'мок вполне', 'максимально учтены', 'ледникам возможен', 'которое прошло', 'итогам совещания', 'искусственно завышенной', 'интересы собственников', 'глава оргкомитета', 'выкуп земли', 'возведении объектов', 'вайншток отметил', 'бобслейной трассы', 'словам путина', 'словам козака', '« это', '« скорее', 'участников совещания', 'суд ».', 'региональные власти', 'подготовке олимпиады', 'словам', '

['эта мера', 'экономический рост', 'численность иждивенцев', 'человеческий капитал', 'целом зарплаты', 'федерального бюджета', 'уровню расходов', 'уровень смертности', 'тесном контакте', 'теневой сектор', 'стопроцентной гарантии', 'службе минздрава', 'руб .».', 'решение связано', 'развитых странах', 'развитых стран', 'развитых государств', 'равный 102', 'пытаются опровергнуть', 'профильных ведомств', 'профильным ведомствам', 'прогнозе внешэкономбанка', 'преподносят чиновники', 'посвященную здравоохранению', 'повышению зарплат', 'период увеличилась', 'пенсионным возрастом', 'пенсионной системе', 'официально власти', 'опубликовал доклад', 'оказываемой гражданам', 'образованных кадров', 'никаких прогнозов', 'необходимо увеличение', 'некоторые министры', 'нашли поддержки', 'нашем варианте', 'настоящая проблема', 'напрямую влияет', 'молодого возрастов', 'минздрав откликнулся', 'коэффициенты показывают', 'который начинается', 'институте подчеркивают', 'заявила вице', 'других странах', 'демог

['являющийся депутатом', 'четыре созыва', 'хороших прогнозов', 'также госдуму', 'стать грудинин', 'сказал закондырин', 'региональной элите', 'президентской кампании', 'президентских выборов', 'пока такого', 'позиции отнюдь', 'подчеркнул источник', 'подмосковных округов', 'партии роста', 'нынешней кампании', 'московского уровня', 'мнению политика', 'менее известный', 'которые состоятся', 'кого выдвигаться', 'кого выдвигать', 'касается кандидатов', 'кандидатов дижура', 'другими партиями', 'дают оснований', 'высшие должности', 'выдвинула оппозиция', 'воробьев идет', 'включая кпрф', 'весьма вероятно', 'администрации президента', '18 марта', 'избирается ».', '« кого', '« единственный', 'этих партий', 'чеховском районах', 'участия оппозиции', 'регионе политик', 'ранее баллотировавшийся', 'президентских выборах', 'пр ).', 'павла грудинина', 'нынешнем сентябре', 'борис надеждин', 'подмосковье ».', '».', '«', 'сентябре', 'ранее', 'районах', 'пр', 'политик', 'подмосковье', 'партий', 'оппозиции',

['этих странах', 'этими альтернативами', 'электоральных демократий', 'шансом построить', 'чаще рассматриваться', 'хх веке', 'хронической жертвой', 'утверждает правду', 'условиях демократий', 'упрощенно говоря', 'университетской среде', 'украиной управление', 'удержание власти', 'трансформации происходили', 'традиционных площадок', 'травматического состояния', 'технократического управления', 'тех политических', 'тема разговора', 'таких случаях', 'таких масштабах', 'таким образом', 'сфер жизни', 'сумели приподняться', 'стремятся выбрать', 'страшно подумать', 'странах капитализма', 'стала поспокойнее', 'стал ссылаться', 'сравнимых кейсов', 'способные влиять', 'сообществом чувства', 'сообществом интересов', 'сообщества бессильны', 'сообщении чувствовался', 'совершенно бесчувственна', 'снова обиженные', 'следующий день', 'сильными искажениями', 'сила нравственности', 'серьезный разговор', 'сергей аверинцев', 'своем городе', 'свободных дискуссий', 'самодовольной интеллигентщины', 'сайте geft

0      [функцию организатора, федеральном уровне, учебных заведений, убийства закрыто, тех регионов, такую ответственность, судебный процесс, стороны силовиков, стараемся координировать, становимся кандидатами, сих пор, российской экономике, различных сми, проходят лекции, провести акцию, предлагать взять, права женщин, поступали угрозы, поставленными задачами, пассивное наблюдение, открытость политики, освобождение политзаключенных, огромное влияние, обсуждения социально, образом действовать, неподдельная заинтересованность, необходимо каким, некоторые пытаются, некоторое время, нашу форму, наступления консерватизма, наемные работники, московских активистах, московские активисты, многие интересуются, многие боятся, мнению одного, либеральной альтернативы, контрактную армию, команде единомышленников, избегая насилия, идеологически силе, заканчивая школу, единого руководства, должен присоединиться, деятельности партии, день борьбы, готовы взять, городской повестке, выполнить взятые, воз

**Эксперимент 4**  
Количество ключевых слов

**Удается получить 0.17, если сокращаем количество самых частотных слов до 7**, что является сомнительным достижением

In [79]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(7)]))

Precision -  0.15
Recall -  0.21
F1 -  0.17
Jaccard -  0.1


**Результат 4: немного выше бейслайна, но это, скорее, хак метрики.**

In [82]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.2
Recall -  0.13
F1 -  0.14
Jaccard -  0.09


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [23]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                         [яблоко, молодёжный, который, акция, год, активист, это, деятельность, политика, наш]
1                               [миллиард, газа, год, куб, метр, газпром, добыча, 2020, должный, производитель]
2                                              [год, это, книга, роман, тот, писать, выйти, один, мир, перевод]
3                                    [ким, зинаида, видео, год, журналист, суд, дело, бывший, футиный, который]
4                         [площадь, территория, новый, москва, га, который, столица, тинао, парковый, развитие]
5                                  [гонка, который, команда, место, позиция, два, один, из-за, круг, чемпионат]
6                        [место, влияние, рф, позиция, глава, россия, президент, сергей, политический, рейтинг]
7                 [культура, закон, который, сфера, стд, разработать, концепция, проект, изменение, сообщество]
8                              [газопровод, сирия, год, турция, газа, россия, европа, катар, который, ту

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

**Эксперимент 5**  
**Извлекаем устойчивые коллокации из текста**

In [83]:
import nltk
from nltk.collocations import *
ignored_words = nltk.corpus.stopwords.words('russian')
bigram_measures = nltk.collocations.BigramAssocMeasures()

def getcollocations(text):
    proccolls = []
    tokens = nltk.wordpunct_tokenize(text)
    toklem  = [morph.parse(word)[0].normal_form for word in tokens]
    finder = BigramCollocationFinder.from_words(toklem, window_size = 2)
    finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
    finder.apply_freq_filter(1)
    colls = finder.nbest(bigram_measures.likelihood_ratio, 10)
    for a in colls:        
        p = morph.parse(a[1])[0]        
        if p and p.tag.gender and 'NOUN' in p.tag:
            p0 = morph.parse(a[0])[0]
            if p0 and 'ADJF' in p0.tag:
                nf  = p0.inflect({'sing', 'nomn', p.tag.gender})
                if nf:
                    proccolls.append(nf.word +" " + a[1])
    return(proccolls)

In [87]:
# data['content_norm'] = data['content'].apply(normalize)
data['content_colloc'] = data['content'].apply(getcollocations)

In [88]:
data['title_up'] = data['content_colloc']+data['title_norm']

In [89]:
# data["content_colloc"]
evaluate(data['keywords'],data['title_up'].apply(lambda x: x[:10]))

Precision -  0.12
Recall -  0.16
F1 -  0.13
Jaccard -  0.08


**Результат 5: неудачно**


**Эксперимент 6**  
Отбор слов по части речи

In [32]:
def normalizepos(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in ['NOUN']]

    return words

In [33]:
data['content_norm'] = data['content'].apply(normalizepos)

In [34]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


попробуем оставить не только существительные, но и прилагательные

In [35]:
def normalize4(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in ['NOUN', 'ADJF']]

    return words

In [36]:
data['content_norm4'] = data['content'].apply(normalize4)

In [37]:
evaluate(data['keywords'], data['content_norm4'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.23
F1 -  0.15
Jaccard -  0.08


**Результат 6: неудачно**

**Эксперимент 7**  
Попробуем дополнительно выбросить слова, которые часто повторяются, но не оказываются ключевыми

In [39]:
data['content_norm4'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                       [яблоко, молодёжный, который, акция, год, активист, деятельность, политика, наш, московский]
1                                  [миллиард, газа, год, куб, метр, газпром, добыча, производитель, который, страна]
2                                        [год, книга, роман, тот, один, мир, перевод, стихотворение, читатель, жанр]
3                                         [ким, зинаида, видео, год, журналист, суд, дело, бывший, футиный, который]
4                                 [площадь, территория, новый, москва, га, который, столица, тинао, развитие, парка]
5                                       [гонка, который, команда, место, позиция, два, один, круг, чемпионат, пилот]
6                             [место, влияние, рф, позиция, глава, россия, президент, сергей, политический, рейтинг]
7                  [культура, закон, который, сфера, концепция, проект, изменение, сообщество, услуга, общественный]
8                                   [газопровод, сирия, год, тур

In [56]:
morestops = {"год", "страна", "главный", "развитый", "март", "сегодня", "бумажный", "нынешний", "назад", "популярный"} | stops
def normalize23(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word]
    words = [word.normal_form for word in words if word.tag.POS in ['NOUN', 'ADJF'] and word.normal_form not in morestops]

    return words

In [57]:
data['content_norm23'] = data['content'].apply(normalize23)

In [58]:
evaluate(data['keywords'], data['content_norm23'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.09


In [59]:
data['content_norm23'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                            [яблоко, молодёжный, который, акция, активист, деятельность, свой, политика, весь, наш]
1                      [миллиард, газа, куб, метр, газпром, добыча, производитель, который, независимый, внутренний]
2                                  [весь, книга, роман, мир, перевод, стихотворение, читатель, жанр, поэзия, работа]
3                                       [ким, зинаида, видео, журналист, суд, дело, бывший, футиный, который, рубль]
4                                 [площадь, территория, новый, москва, га, который, столица, тинао, развитие, парка]
5                            [гонка, который, команда, место, позиция, круг, чемпионат, пилот, последний, следующий]
6                             [место, влияние, рф, позиция, глава, россия, президент, сергей, политический, рейтинг]
7                  [культура, закон, который, сфера, концепция, проект, изменение, сообщество, услуга, общественный]
8                                 [газопровод, сирия, турция, га

**Результат 7**  
Тоже без особого успеха

<hr>

Ещу улучшения!

In [49]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0        [яблоко, акция, год, активист, деятельность, политика, власть, задача, молодая, человек]
1               [миллиард, газа, год, куб, метр, газпром, добыча, производитель, страна, прогноз]
2                [год, книга, роман, мир, перевод, стихотворение, читатель, жанр, поэзия, работа]
3                     [ким, зинаида, видео, год, журналист, суд, дело, рубль, процесс, заседание]
4                   [площадь, территория, москва, га, столица, тинао, развитие, парка, парк, год]
5                    [гонка, команда, место, позиция, круг, чемпионат, пилот, бокс, заезд, льюис]
6               [место, влияние, рф, позиция, глава, россия, президент, сергей, рейтинг, участие]
7    [культура, закон, сфера, концепция, проект, изменение, сообщество, услуга, учреждение, дело]
8                    [газопровод, сирия, год, турция, газа, россия, европа, катар, поток, проект]
9                 [участник, рф, организация, государство, область, центр, связь, фсб, март, год]
Name: content_norm, 

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [90]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [91]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [92]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [93]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [94]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [95]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [96]:
keywords[:3]

[['яблоко',
  'активист',
  'акция',
  'дарья',
  'деятельность',
  'молодая человек',
  'политика',
  'виктор',
  'выборы',
  'силовик'],
 ['миллиард куб',
  'куб метр',
  'куб',
  'газпром',
  'газа',
  'миллиард',
  'добыча',
  'метр',
  'добыча газа',
  'холдинг'],
 ['роман',
  'книга',
  'жанр',
  'стихотворение',
  'перевод',
  'читатель',
  'год',
  'поэзия',
  'произведение',
  'том']]

In [97]:
evaluate(data['keywords'], keywords)

Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

***Эксперимент 8***

**Объединяем результаты отбора по TfIdf с коллокациями**

In [98]:
data["kw"] = keywords

In [99]:
data["both"] = data["kw"]+data["content_colloc"]

In [100]:
evaluate(data['keywords'], data["both"])

Precision -  0.1
Recall -  0.27
F1 -  0.14
Jaccard -  0.08


**Результат 8: неудачно**  


**Эксперимент 9**   
Попробуем поменять параметра TfIdfVectorizer

In [101]:
tfidf2 = TfidfVectorizer(ngram_range=(1,2), min_df=2)
tfidf2.fit(data['content_norm_str'])
id2word2 = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors2 = tfidf2.transform(data['content_norm_str'])
keywords2 = [[id2word2[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 
evaluate(data['keywords'], keywords2)

Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09


**Результат 9: неудачно**

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [46]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [47]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [48]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

Wall time: 1min 31s


In [49]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.13
Jaccard -  0.08


In [50]:
keywords_rw.head(10)

0                       [яблоко, год, власть, акция, активист, молодёжь, политика, россия, убийство, школа]
1                       [год, миллиард, газа, газпром, добыча, метр, производитель, куб, население, страна]
2                                   [книга, роман, год, идея, рука, стих, удовольствие, жанр, мир, перевод]
3                     [ким, экспертиза, зинаида, видео, дело, журналист, заседание, монтаж, склейка, слово]
4                               [площадь, парка, га, территория, москва, парк, развитие, регион, зона, год]
5                                [гонка, команда, позиция, пилот, бокс, место, сезон, сутил, трасса, заезд]
6                            [место, влияние, рф, позиция, глава, президент, сергей, число, лидер, участие]
7    [культура, закон, учреждение, концепция, проект, союз, законодательство, система, разработка, человек]
8                       [турция, газопровод, россия, катар, поток, европа, сирия, газа, строительство, год]
9                       [уча

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [51]:
import networkx as nx

In [52]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=3, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.pagerank(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [53]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

Wall time: 2min 32s


In [54]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09


Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

Готовое решение есть в gensim. Давайте попробуем его.

In [55]:
from gensim.summarization import keywords

c:\python36\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [56]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])

In [57]:
evaluate(data['keywords'], gensim_kws)

Precision -  0.07
Recall -  0.11
F1 -  0.08
Jaccard -  0.04


Наша имплементация отработала получше.

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/H9lBH9wCxqq1T0ru2

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).